In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

In [34]:
# Import findspark and initialize.
import findspark
findspark.init()

In [35]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [36]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [38]:
# 2. Create a temporary view of the DataFrame.
import requests

response = requests.get(url)
with open("home_sales.csv", "wb") as file:
    file.write(response.content)

df = spark.read.csv("home_sales.csv", header=True, inferSchema=True)
df.createOrReplaceTempView("home_sales")




In [41]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

result = spark.sql("""SELECT
        SUBSTRING(date, 1, 4) AS sale_year,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales
    WHERE
        bedrooms = 4
    GROUP BY
        SUBSTRING(date, 1, 4)
    ORDER BY
        sale_year""")


result.show()


+---------+---------+
|sale_year|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [42]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

result = spark.sql("""SELECT
        SUBSTRING(date_built, 1, 4) AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales
    WHERE
        bedrooms = 3 AND bathrooms = 3
    GROUP BY
        SUBSTRING(date_built, 1, 4)
    ORDER BY
        year_built""")


result.show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [43]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

result = spark.sql("""SELECT
        SUBSTRING(date_built, 1, 4) AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales
    WHERE
        bedrooms = 3 AND
        bathrooms = 3 AND
        floors = 2 AND
        sqft_living >= 2000
    GROUP BY
        SUBSTRING(date_built, 1, 4)
    ORDER BY
        year_built""")


result.show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [44]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00020599365234375 seconds ---


In [45]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [46]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [47]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 0.0001404285430908203 seconds ---


In [48]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

partitioned_parquet_path = "/content/partitioned_home_sales_parquet"

df.write.partitionBy("date_built").parquet(partitioned_parquet_path)


In [49]:
# 11. Read the formatted parquet data.

partitioned_df = spark.read.parquet(partitioned_parquet_path)


partitioned_df.printSchema()
partitioned_df.show()


root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- date_built: integer (nullable = true)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         

In [50]:
# 12. Create a temporary table for the parquet data.
partitioned_df.createOrReplaceTempView("partitioned_home_sales")

In [51]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00015234947204589844 seconds ---


In [52]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [53]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")

if is_cached:
    print("Yes")
else:
    print("No")


No
